# <font color="Blue">Abstract</font>

In this notebook, we delve into the utilization of LangChain and its 'prompt template' function to create prompt for giving interview questions.

**<font color="red">Our goal</font>** is to craft an effective prompt tailored to providing interview questions in the context of specific jobs and companies.  

We used following knowledge:  
Prompt Engineering Skills:  
1. Placeholder;
2. Persona.


<font color="green"> Conclusion: </font>  
Through this notebook, we found that at least for Amazon and S&P Global, which are famous companies, do not need detailed information to get good interview questions.  
In my opinion,  
1. for those big companies, we can only use their [company names] and the [role title] that we are applying.  
2. maybe for some small companies, users can provide [company description] that they found might help them prepare the job interview.  
3. for any company, if users know some information, like Amazon Leadership Principles, they could mention specifically in [company description] in case the model did not pick up by themselve.

# Prepare the notebook for use

## Install packages & import libraries

In [1]:
# Download required packages to Colab
!pip install -q langchain
!pip install -q openai
!pip install -q chromadb
!pip install -q tiktoken
!pip install -q duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.8/496.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.4 MB/s eta

In [2]:
# Import required libraries
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain import ConversationChain
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import os
from getpass import getpass

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

import pandas as pd
import csv

## OpenAI API key

In [3]:
openai_api_key = getpass()
os.environ["OPENAI_API_KEY"] = openai_api_key

··········


## Connect Google Drive

## Create .csv files to collect info from this notebook

Since the results we got from LLMs will be different everytime, we want to use .csv files to track those results which could help use go back to see what we can do or what we did to improve the outputs.

In [4]:
# with open('/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv', 'w', newline='') as csvfile:
#     fieldnames = ['comb_num','interviewer_role','company_name','resume','role_title','role_description','company_description']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

#     writer.writeheader()
var_comb = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv")

In [29]:
with open('/content/drive/MyDrive/Capstone Project - Mock Interview App/interview_questions_prompt_result.csv', 'w', newline='') as csvfile:
    fieldnames = ['model', 'variable_combination', 'template', 'output']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    #writer.writerow({'input': user_input, 'output': result})

In [30]:
prompt_result = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/interview_questions_prompt_result.csv")

In [8]:
var_comb

,comb_num,interviewer_role,company_name,resume,role_title,role_description,company_description
0,1,recruiter,S&P Global,\nEDUCATION\nVanderbilt University – Data Scie...,Data Engineer - AI/ML (Virtual),\nJob Description\n\nThis is a software develo...,\nOverview\nS&P Global (NYSE: SPGI) provides e...


In [31]:
prompt_result

,model,variable_combination,template,output


# Model selection

In [15]:
chat = ChatOpenAI(model_name = "gpt-4")

In [26]:
# record model as ex_model
ex_model = "gpt_4"

# Interview question prompts

## Prompt 1 & 2

### Extra info

Two days ago, I explored to just use [COMPANY NAME] and [JOB TITLE] to test questions will be asked to interview _Amazon_ and _data science_ job.

The following are the conversation between GPT and me:

<font color='Green'>ME:</font>  
Help me 1) rewrite my prompt for you to make you give me a better result; 2) generate the result for your prompt. And give yourself a score and rationales of why it is better for you to give better results.  To do this task, first, generate results from my prompt; Then start the rewrite task that I describe above.

***My prompt: I am preparing a job interview for [COMPANY NAME: Amazon] [JOB TITLE: data scientist] job. You as an [INTERVIEWER ROLE: recruiter], give me 10 interview questions that you are most likely to ask me to check whether I am a fit candidate.***


<font color='Blue'>GPT:</font>   
First, let's generate the result from your original prompt:

Interview Questions for a Data Scientist Position at Amazon:

1. Can you describe a challenging data analysis project you've worked on? How did you approach it and what was the outcome?
2. How do you stay updated with the latest trends and tools in data science?
3. Describe a time when you had to explain complex data to a non-technical audience. How did you ensure they understood?
4. How would you handle missing or corrupted data in a dataset?
5. Can you explain the difference between supervised and unsupervised machine learning?
6. Describe your experience with cloud computing platforms, like AWS.
7. What programming languages are you proficient in, and how have you applied them in your data science projects?
8. How do you ensure the accuracy of your data analysis?
9. Can you give an example of how you've used data visualization to help make a business decision?
10. What are your thoughts on the ethical implications of data collection and analysis?

*Rewriting Your Prompt for Enhanced Results*:  
**"I am preparing for a data scientist job interview at Amazon. As a professional with expertise in recruitment and interview processes for technical roles, can you provide me with specific and challenging interview questions that Amazon might ask to assess my technical and problem-solving skills, familiarity with Amazon's work culture, and ability to handle real-world data science scenarios?"**

Generated Result for Rewritten Prompt:

1. How would you apply Amazon's leadership principles to a data science project?
2. Describe a project where you used machine learning to solve a real-world problem. What was the impact?
3. Explain how you would use AWS tools in a data science workflow.
4. How do you approach a new dataset? Walk us through your process from data cleaning to analysis.
5. Can you discuss a time when you had to make a critical decision based on incomplete data?
6. Describe your experience with big data technologies and how you’ve utilized them in past projects.
7. How do you balance the need for accurate, high-quality data with time constraints in a fast-paced environment?
8. Can you give an example of a time when you had to innovate to solve a data-related challenge?
9. How do you ensure that your models are not only accurate but also fair and unbiased?
10. Explain a complex data science concept in a way that would be understandable to someone without a technical background.

Score and Rationale:
Score: 8/10

The rewritten prompt directs towards more specific and relevant questions that align closely with Amazon's unique work culture and the practical challenges a data scientist might face. The focus on Amazon's leadership principles, real-world applications, and the use of specific tools like AWS, makes the questions more targeted and reflective of what Amazon might prioritize in their interview process. This tailoring leads to a more useful and realistic set of questions for interview preparation.


<font color='Green'>ME:</font>  
Redo the rewrite task again, this time generate a 9/10 score results


<font color='Blue'>GPT:</font>  
To refine your prompt further for an even more targeted set of interview questions for a Data Scientist role at Amazon, let's incorporate elements that focus on Amazon's specific business challenges, technological environment, and its emphasis on innovation and customer orientation. Here's the revised prompt:

*Revised Prompt for Enhanced Results*:  
**"I am gearing up for a Data Scientist interview at Amazon. Given your deep understanding of Amazon's unique business challenges, technological environment, and its focus on customer-centric solutions, could you provide advanced and specific interview questions that would assess my technical expertise, problem-solving abilities, and alignment with Amazon's mission and principles, particularly in data-driven decision-making and innovation in the field of data science?"**

Generated Result for the Revised Prompt:

1. How would you use data science to enhance the customer experience on Amazon's platforms?
2. Amazon values innovation; can you discuss a project where you innovated a new approach or solution in data science?
3. Given Amazon's vast and varied datasets, how would you approach a project that involves integrating data from multiple, disparate sources?
4. Describe how you would use machine learning to improve the efficiency of Amazon's logistics and supply chain.
5. How would you apply data science to predict and meet future customer demands in a specific market segment?
6. Can you provide an example of how you would use data analysis to support Amazon's commitment to sustainability?
7. How do you approach the challenge of balancing data privacy with data utility in a consumer-centric business like Amazon?
8. Describe your experience with AWS services in data science projects. How did these tools help you achieve your project goals?
9. Can you explain a time when you had to adjust your model or approach based on feedback or changing business needs?
10. How would you leverage data science to identify new business opportunities or potential areas of growth for Amazon?

Score and Rationale:
Score: 9/10

This revised prompt and the resulting questions are tailored to not only assess general data science skills but also focus specifically on areas critical to Amazon, like customer experience, innovation, and handling complex, large-scale data scenarios. The questions are formulated to gauge an understanding of Amazon's business model and its emphasis on customer-centricity, making them highly relevant and challenging for a candidate preparing for an Amazon data scientist interview. This level of specificity and alignment with Amazon's core principles makes the response more effective for interview preparation.

<font color='Green'>ME:</font>  
Could you score my prompt and give rationale?

<font color='Blue'>GPT:</font>  
Score for Your Original Prompt:
Score: 6/10

Rationale:
Your original prompt was clear and straightforward, asking for interview questions for a data scientist position at Amazon. This is effective in prompting a general set of relevant interview questions. However, the prompt could be enhanced by specifying certain aspects unique to Amazon, such as their leadership principles, the emphasis on customer-centricity, and the kind of technological challenges specific to Amazon's data science roles.

While the prompt successfully solicits a range of typical questions a recruiter might ask, it lacks the depth to extract questions that deeply align with the unique aspects of Amazon's work culture and the specific challenges a data scientist might face there. By incorporating these elements, the prompt could yield a set of questions that would be more beneficial for a candidate specifically preparing for an interview with Amazon. This refinement would lead to a more targeted and thorough preparation, thus enhancing the overall effectiveness of the response.


**HERE** <font color='Blue'> compares my original prompt and GPT prompt (gpt4-turbo):</font>  

<font color='green'>My Prompt: [6/10]</font>   
I am preparing a job interview for [COMPANY NAME: Amazon] [JOB TITLE: data scientist] job. You as an [INTERVIEWER ROLE: recruiter], give me 10 interview questions that you are most likely to ask me to check whether I am a fit candidate.

<font color='green'>GPT Prompt: [8/10]</font>   
I am preparing for a data scientist job interview at Amazon. As a professional with expertise in recruitment and interview processes for technical roles, can you provide me with specific and challenging interview questions that Amazon might ask to assess my technical and problem-solving skills, familiarity with Amazon's work culture, and ability to handle real-world data science scenarios?

<font color='green'>GPT Prompt: [9/10]</font>   
I am gearing up for a Data Scientist interview at Amazon. Given your deep understanding of Amazon's unique business challenges, technological environment, and its focus on customer-centric solutions, could you provide advanced and specific interview questions that would assess my technical expertise, problem-solving abilities, and alignment with Amazon's mission and principles, particularly in data-driven decision-making and innovation in the field of data science?

**HERE** <font color='Blue'> are rationales about the score that ChatGPT4 gave about my original prompt and GPT prompt (gpt4-turbo):</font>  

<font color='green'>My Prompt: [6/10]</font>   
Your original prompt was clear and straightforward, asking for interview questions for a data scientist position at Amazon. This is effective in prompting a general set of relevant interview questions. However, the prompt could be enhanced by specifying certain aspects unique to Amazon, such as their leadership principles, the emphasis on customer-centricity, and the kind of technological challenges specific to Amazon's data science roles.

<font color='green'>GPT Prompt: [8/10]</font>   
The rewritten prompt directs towards more specific and relevant questions that align closely with Amazon's unique work culture and the practical challenges a data scientist might face. The focus on Amazon's leadership principles, real-world applications, and the use of specific tools like AWS, makes the questions more targeted and reflective of what Amazon might prioritize in their interview process. This tailoring leads to a more useful and realistic set of questions for interview preparation.

<font color='green'>GPT Prompt: [9/10]</font>   
This revised prompt and the resulting questions are tailored to not only assess general data science skills but also focus specifically on areas critical to Amazon, like customer experience, innovation, and handling complex, large-scale data scenarios. The questions are formulated to gauge an understanding of Amazon's business model and its emphasis on customer-centricity, making them highly relevant and challenging for a candidate preparing for an Amazon data scientist interview. This level of specificity and alignment with Amazon's core principles makes the response more effective for interview preparation.

### Code

In [19]:
# set up prameters
job_title = var_comb.role_description[0]
comp_name = var_comb.company_name[0]

In [20]:
# set up input variables for prompt
input_var_1 = ["role_title", "company_name"]

In [24]:
# record those parameters as ex_var_comb
ex_var_comb = 1 # this means we use the 1st combination of information in var_comb.csv dataset; this helps us tack information

#### Prompt with score 8/10

In [32]:
template_1 = """
I am preparing for a {role_title} job interview at {company_name}. As a professional
with expertise in recruitment and interview processes for technical roles, can
you provide me with specific and challenging interview questions that {company_name}
might ask to assess my technical and problem-solving skills, familiarity with
{company_name}'s work culture, and ability to handle real-world data science scenarios?
"""

In [33]:
prompt_1 = PromptTemplate(
    input_variables = input_var_1,
    template = template_1
)

new_prompt_1 = prompt_1.format(role_title=job_title,
                             company_name=comp_name)
result_1 = chat([HumanMessage(content=new_prompt_1)])

In [34]:
print(result_1.content)

1. Can you describe a time when you had to design an AI/ML model from scratch to solve a business problem? What were the challenges and how did you overcome them?

2. How would you explain Machine Learning to a non-technical stakeholder? 

3. Can you discuss a time when you had to clean and prepare a large dataset for a project? What were the challenges and how did you handle them?

4. S&P Global works in a fast-paced environment. Can you provide an example illustrating your ability to quickly adapt to new technologies or methodologies?

5. Can you share your experience working in an Agile environment? What do you consider to be the key advantages and challenges of the Agile approach?

6. Describe a situation where you had to communicate a complex technical concept to a non-technical audience. How did you ensure that the audience understood your explanation?

7. Can you provide a specific example of when you had to troubleshoot a problem in your code? What was the problem and how did y

##### store output

In [36]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [ex_var_comb],
    'template': [template_1],
    'output': [result_1.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [37]:
prompt_result

,model,variable_combination,template,output
0,gpt_4,1,\nI am preparing for a {role_title} job interv...,1. Can you describe a time when you had to des...


#### Prompt with score 9/10

In [40]:
template_2 = """
I am gearing up for a {role_title} interview at {company_name}. Given your deep
understanding of {company_name}'s unique business challenges, technological environment,
and its focus on customer-centric solutions, could you provide advanced and
specific interview questions that would assess my technical expertise,
problem-solving abilities, and alignment with {company_name}'s mission and principles,
particularly in data-driven decision-making and innovation in the field of data
science?
"""

In [42]:
prompt_2 = PromptTemplate(
    input_variables = input_var_1,
    template = template_2
)

new_prompt_2 = prompt_2.format(role_title=job_title,
                             company_name=comp_name)
result_2 = chat([HumanMessage(content=new_prompt_2)])

In [43]:
print(result_2.content)

1. Can you share an example of a project where you developed AI or ML models to help a business make better decisions? What was the outcome?

2. How have you used programming languages such as Python or Java in your previous roles to develop software solutions?

3. S&P Global relies heavily on data-driven decisions. Can you tell us about a time when you used data to solve a complex problem?

4. We often encounter situations where data is unstructured or messy. Can you describe your experience in data collection and cleaning? 

5. Explain how you evaluate the performance of AI/ML models. Can you provide an example of a time when you had to make adjustments to a model based on your evaluation?

6. Communication and collaboration are key in this role. Can you tell us about a time when you worked with stakeholders to understand business needs and develop solutions?

7. Can you discuss a time when you worked closely with a QA team to fix bugs? How did you handle communication and collaborat

##### store output

In [44]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [ex_var_comb],
    'template': [template_2],
    'output': [result_2.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [45]:
prompt_result

,model,variable_combination,template,output
0,gpt_4,1,\nI am preparing for a {role_title} job interv...,1. Can you describe a time when you had to des...
1,gpt_4,1,\nI am gearing up for a {role_title} interview...,1. Can you share an example of a project where...


## Prompt 3 & 4 & 5

### Extra info

This time we have all variables in var_comb.csv.

My conversation with ChatGPT4 is:

<font color='Green'>ME:</font>  
Now, I have [interviewer_role], [company_name], [resume], [role_title], [role_description], and [company_description] as placeholders.  
[interviewer_role] can be: a recruiter, hiring manager, senior data scientist, or whoever interviews me
[resume] is my resume
[role_title] is the job title that  I want to apply
[role_desctiption] is the job description in the job post for [role_title]
[company_desctiption] is the short description of the [company_name] from LinkedIn

Could you help me generate 3 prompt templates including the above placeholders? Those templates will generate 10 high-quality interview questions that interviewers are highly likely to ask during the job interview.

<font color='Blue'>GPT:</font>  
Certainly! Here are three prompt templates incorporating your placeholders, designed to generate high-quality interview questions for your job interview:

**Template 1: General Fit and Skills Assessment**
"As an [interviewer_role] at [company_name], I'm looking to assess the suitability of a candidate for [role_title]. Based on [resume] and considering the responsibilities listed in [role_description], what are 10 insightful questions you would ask to evaluate the candidate's skills, experience, and overall fit for the role and company culture detailed in [company_description]?"

**Template 2: Technical and Problem-Solving Focus**
"I am a [interviewer_role] preparing to interview a candidate for the [role_title] at [company_name]. Given their technical background as highlighted in [resume] and the specific technical requirements mentioned in [role_description], what are 10 challenging technical and problem-solving questions that would effectively gauge their technical proficiency and ability to contribute to the objectives outlined in [company_description]?"

**Template 3: Behavioral and Scenario-Based Questions**
"As an [interviewer_role] for [company_name], I am interested in understanding how a candidate for [role_title] would fit into our team dynamics and handle real-world scenarios. Referring to their experiences listed in [resume] and the key competencies required as per [role_description], could you suggest 10 behavioral and scenario-based interview questions that align with [company_description] and help assess the candidate's soft skills, decision-making abilities, and alignment with our company values?"

These templates are designed to elicit specific, relevant, and high-quality interview questions from a variety of interviewer perspectives, focusing on different aspects of the candidate's qualifications and fit for the role and company.



### Code

In [47]:
# set up prameters
job_title = var_comb.role_description[0]
comp_name = var_comb.company_name[0]
interviewer_title = var_comb.interviewer_role[0]
my_resume = var_comb.resume[0]
role_des = var_comb.role_description[0]
comp_des = var_comb.company_description[0]

In [48]:
# set up input variables for prompt
input_var_2 = ["interviewer_role", "company_name", "role_title", "resume", "role_description", "company_description"]

#### Prompt 3: general fit and skills assessment

In [49]:
template_3 = """
As an {interviewer_role} at {company_name}, I'm looking to assess the suitability
 of a candidate for {role_title}. Based on {resume} and considering the
 responsibilities listed in {role_description}, what are 10 insightful questions
  you would ask to evaluate the candidate's skills, experience, and overall fit
  for the role and company culture detailed in {company_description}?
"""

prompt_3 = PromptTemplate(
    input_variables = input_var_2,
    template = template_3
)

new_prompt_3 = prompt_3.format(role_title=job_title,
                               company_name=comp_name,
                               interviewer_role=interviewer_title,
                               resume=my_resume,
                               role_description=role_des,
                               company_description=comp_des)
result_3 = chat([HumanMessage(content=new_prompt_3)])

In [50]:
print(result_3.content)

1. Can you share an example of a project where you implemented machine learning algorithms and the impact it had on the project?
2. How have you managed the process of data collection and cleaning in your past roles or projects?
3. Can you provide an example of how you evaluated the performance of an AI/ML model and what actions you took as a result of the evaluation?
4. How have you worked with stakeholders to understand their business needs and developed solutions accordingly?
5. Could you describe a situation where you faced a challenge while deploying an AI model? How did you resolve it?
6. Can you share your experience with daily scrum updates and how you contributed to the process?
7. How have you worked closely with a QA team to fix bugs? Can you provide a specific example?
8. What is your understanding of automation in the context of software development? Can you share an example of when you applied this understanding in a project?
9. How do you keep yourself updated with emerg

##### store output

In [54]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [ex_var_comb],
    'template': [template_3],
    'output': [result_3.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

#### Prompt 4: technical and problem-solving focus

In [51]:
# set up input variables for prompt
input_var_3 = ["interviewer_role", "role_title", "company_name", "resume", "role_description", "company_description"]

In [52]:
template_4 = """
I am a {interviewer_role} preparing to interview a candidate for the {role_title}
 at {company_name}. Given their technical background as highlighted in {resume}
 and the specific technical requirements mentioned in {role_description},
 what are 10 challenging technical and problem-solving questions that would
 effectively gauge their technical proficiency and ability to contribute to the
 objectives outlined in {company_description}?
"""

prompt_4 = PromptTemplate(
    input_variables = input_var_3,
    template = template_4
)

new_prompt_4 = prompt_4.format(role_title=job_title,
                               company_name=comp_name,
                               interviewer_role=interviewer_title,
                               resume=my_resume,
                               role_description=role_des,
                               company_description=comp_des)
result_4 = chat([HumanMessage(content=new_prompt_4)])

In [53]:
print(result_4.content)

1. Can you describe a machine learning project you've worked on, the challenges you faced, and how you overcame them?
2. How would you go about implementing a new AI model in an existing system? What considerations would you need to take into account?
3. Can you explain how you would evaluate the performance of an AI/ML model? What metrics would you use and why?
4. How would you approach cleaning a large dataset with missing or inconsistent data?
5. Could you describe a situation where you had to work closely with stakeholders to understand their needs and develop a solution?
6. How would you handle a situation where the AI model you've developed is not performing as expected?
7. Can you explain your process for fixing bugs and ensuring the quality of your code?
8. Describe a time when you had to make a crucial decision in a project. What was the situation, and how did you handle it?
9. Have you ever had to learn a new technology or programming language for a project? How did you go ab

##### store output

In [55]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [ex_var_comb],
    'template': [template_4],
    'output': [result_4.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

#### Prompt 5: behavioral and scenario-based questions

In [56]:
input_var_2

['interviewer_role',
 'company_name',
 'role_title',
 'resume',
 'role_description',
 'company_description']

In [57]:
template_5 = """
As an {interviewer_role} for {company_name}, I am interested in understanding
how a candidate for {role_title} would fit into our team dynamics and handle
real-world scenarios. Referring to their experiences listed in {resume} and the
key competencies required as per {role_description}, could you suggest 10
behavioral and scenario-based interview questions that align with
{company_description} and help assess the candidate's soft skills, decision-making
 abilities, and alignment with our company values?
"""

prompt_5 = PromptTemplate(
    input_variables = input_var_2,
    template = template_5
)

new_prompt_5 = prompt_5.format(role_title=job_title,
                               company_name=comp_name,
                               interviewer_role=interviewer_title,
                               resume=my_resume,
                               role_description=role_des,
                               company_description=comp_des)
result_5 = chat([HumanMessage(content=new_prompt_5)])

In [59]:
print(result_5.content)

1. Can you describe a time when you had to make a decision that wasn't popular but was necessary for a project's success? How did you handle the situation?

2. Tell us about a time when you had to use your strong problem-solving skills to overcome a challenge in a project. What was the situation and what did you do?

3. Our company values innovation and learning. Can you share a situation where you learned a new technology or methodology and applied it to your work?

4. How would you handle a situation where you and a stakeholder have a disagreement about the direction of a project? 

5. In this role, you'll be working closely with a team. Can you describe a time when you had to navigate a conflict within your team? 

6. Describe a time when you had to explain a complex data analysis or technical concept to a non-technical stakeholder. How did you ensure they understood?

7. If you discovered a significant error in an AI/ML model that had already been deployed, how would you handle the

##### store output

In [60]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [ex_var_comb],
    'template': [template_5],
    'output': [result_5.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

## Prompt 6 (comb of 3,4,5)

### Extra info

<font color='Green'>ME:</font>  
Could you combine those three templates into one template? This template can do general fit and skills assessment, technical and problem-solving focus, and behavior and scenario-base questions.

<font color='Blue'>GPT:</font>  
Certainly, I can combine the three templates into a single comprehensive template. Here's the unified template:

**Unified Template for Comprehensive Interview Question Generation:**  
"As an [interviewer_role] interviewing for the [role_title] at [company_name], I need to comprehensively assess a candidate's fit for the position. With reference to the candidate's [resume], the technical requirements and responsibilities in the [role_description], and aligning with the ethos and objectives of [company_description], what are 10 diverse and high-quality interview questions that cover general fit and skills assessment, technical expertise and problem-solving abilities, as well as behavioral and scenario-based competencies? These questions should help in evaluating the candidate's overall suitability for the role in terms of skills, experience, and cultural fit within [company_name]."

This template is designed to prompt questions that holistically evaluate a candidate across various dimensions: their skill set and experience, technical and problem-solving capabilities, and their alignment with the behavioral and cultural aspects of the role and company. This approach ensures a well-rounded assessment of the candidate's potential for the position.

### Code

In [62]:
input_var_3

['interviewer_role',
 'role_title',
 'company_name',
 'resume',
 'role_description',
 'company_description']

In [65]:
template_6 = """
As an {interviewer_role} interviewing for the {role_title} at {company_name},
I need to comprehensively assess a candidate's fit for the position. With
reference to the candidate's {resume}, the technical requirements and
responsibilities in the {role_description}, and aligning with the ethos and
objectives of {company_description}, what are 10 diverse and high-quality
interview questions that cover general fit and skills assessment, technical
expertise and problem-solving abilities, as well as behavioral and scenario-based
competencies? These questions should help in evaluating the candidate's overall
suitability for the role in terms of skills, experience, and cultural fit within
 {company_name}.
"""

prompt_6 = PromptTemplate(
    input_variables = input_var_3,
    template = template_6
)

new_prompt_6 = prompt_6.format(role_title=job_title,
                               company_name=comp_name,
                               interviewer_role=interviewer_title,
                               resume=my_resume,
                               role_description=role_des,
                               company_description=comp_des)
result_6 = chat([HumanMessage(content=new_prompt_6)])

In [66]:
print(result_6.content)

1. General Fit and Skills Assessment Questions:
   - Can you tell me about a time when you had to make a difficult decision within a project? What was the decision and what was the outcome?
   - Given your background in data science, how do you approach problem-solving when you encounter a challenge in your work?
   - How do you ensure that you are continuously learning and staying updated with the latest developments in AI and Machine Learning?

2. Technical Expertise and Problem-Solving Questions:
   - Can you describe a project where you had to implement machine learning algorithms? What were the challenges and how did you overcome them?
   - How have you utilized Python or Java in your past projects to develop and deploy AI models?
   - Can you discuss a time when you had to evaluate the performance of an AI/ML model? What metrics did you use and why?

3. Behavioral and Scenario-based Questions:
   - Can you provide an example of a time when you had to work closely with stakeholder

##### store output

In [69]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [ex_var_comb],
    'template': [template_6],
    'output': [result_6.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

# Save data
interview_questions_prompt_result.csv

In [67]:
var_comb

,comb_num,interviewer_role,company_name,resume,role_title,role_description,company_description
0,1,recruiter,S&P Global,\nEDUCATION\nVanderbilt University – Data Scie...,Data Engineer - AI/ML (Virtual),\nJob Description\n\nThis is a software develo...,\nOverview\nS&P Global (NYSE: SPGI) provides e...


In [70]:
prompt_result

,model,variable_combination,template,output
0,gpt_4,1,\nI am preparing for a {role_title} job interv...,1. Can you describe a time when you had to des...
1,gpt_4,1,\nI am gearing up for a {role_title} interview...,1. Can you share an example of a project where...
2,gpt_4,1,"\nAs an {interviewer_role} at {company_name}, ...",1. Can you share an example of a project where...
3,gpt_4,1,\nI am a {interviewer_role} preparing to inter...,1. Can you describe a machine learning project...
4,gpt_4,1,"\nAs an {interviewer_role} for {company_name},...",1. Can you describe a time when you had to mak...
5,gpt_4,1,\nAs an {interviewer_role} interviewing for th...,1. General Fit and Skills Assessment Questions...


In [71]:
prompt_result.to_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/interview_questions_prompt_result.csv", index=False)